In [4]:
search_for = 124
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002331256866455078
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 621252555
type: [1, 1, 1, 1, 124] 124
cases of this type: 236421376
100000 31.312952624022984
200000 49.60302180466837
300000 46.56602229601254
400000 46.03552002251021
500000 47.93428382093754
600000 46.933724155548724
700000 45.36736660526196
800000 45.663970215455535
900000 45.499920151505194
1000000 16.18609030639132
1100000 46.77924601826518
1200000 45.326819078321336
1300000 43.87974571512687
1400000 44.82993732752384
1500000 47.02421815465644
1600000 45.3946636082326
1700000 44.87557433990335
1800000 45.2509929002422
1900000 39.885053926415395
2000000 22.74033326652075
2100000 45.762131571806755
2200000 44.501764032931106
2300000 41.72392181730922
2400000 46.6679263981846
2500000 42.98794548858904
2600000 44.956027574500304
2700000 42.04730603216609
2800000 42.79565108694972
2900000 19.97900446615626
3000000 41.0439526476

29700000 42.37890108992223
29800000 41.038689599371416
29900000 41.810572307839514
30000000 43.39954223606903
30100000 44.484982095163026
30200000 44.26745600921065
30300000 27.366751163605155
30400000 39.43253979313308
30500000 45.7124351835444
30600000 43.55626387474875
30700000 42.71684242717351
30800000 19.55555225560609
30900000 44.005103768662465
31000000 44.29854475908745
31100000 41.61364564537957
31200000 41.709343873695104
31300000 38.81450640387112
31400000 37.8496268469456
31500000 16.669848125577936
31600000 39.88411790777517
31700000 37.699441436489636
31800000 39.86710264567884
31900000 40.466032451922764
32000000 43.06060477425182
32100000 43.272758321616855
32200000 23.51388191356954
32300000 40.80424499896742
32400000 43.05460336909125
32500000 45.08529811473849
32600000 43.75792094557395
32700000 25.80361760530738
32800000 38.21930515762485
32900000 44.661156279259295
33000000 42.70478889290613
33100000 41.74467505718018
33200000 40.173644503112364
33300000 38.427182

59800000 36.86942963477099
59900000 35.765799717324256
60000000 37.425656022522986
60100000 14.683352239375802
60200000 37.780241149896455
60300000 36.40628718506533
60400000 36.83768309629145
60500000 36.58401569298568
60600000 20.35548409587734
60700000 39.04841730419804
60800000 39.76468529512526
60900000 40.22176393932676
61000000 42.59066585019338
61100000 40.46870089722504
61200000 38.816101048077805
61300000 39.47269026365606
61400000 37.98919398103536
61500000 31.055298280390325
61600000 24.586448931528263
61700000 34.70571358704539
61800000 36.29962389038918
61900000 35.04137885622854
62000000 16.84744844825719
62100000 34.33496802849571
62200000 36.67359648355032
62300000 34.43144795723712
62400000 35.50360083502198
62500000 20.776913676635257
62600000 37.24557075033325
62700000 38.202330378579525
62800000 39.03829609915735
62900000 39.423310010214344
63000000 40.36699675822119
63100000 41.542935805496995
63200000 41.653558199818704
63300000 38.93770061061865
63400000 39.3924

89800000 41.840473215876145
89900000 36.777351558688345
90000000 37.93196615455706
90100000 35.679825816128115
90200000 35.461919293530414
90300000 37.281780629781295
90400000 16.805387978202983
90500000 33.0469767623698
90600000 15.8082468659288
90700000 33.54186260621042
90800000 28.052930276927935
90900000 27.804658774425903
91000000 36.782470826213995
91100000 35.75385934639645
91200000 37.29196901907765
91300000 37.00206763947117
91400000 39.70625950581429
91500000 39.90492455575213
91600000 40.220597320455575
91700000 39.138644319054634
91800000 37.83869163877838
91900000 35.09042630588812
92000000 36.41106456944449
92100000 36.488751170892655
92200000 35.061661637623075
92300000 16.830223506196116
92400000 35.77964066615775
92500000 17.630785029926816
92600000 31.252738552502795
92700000 25.501428797004785
92800000 29.22702267784304
92900000 35.21153135673997
93000000 35.9850801236664
93100000 35.10626665499532
93200000 36.1623729028724
93300000 37.98111844458199
93400000 39.170

119200000 4.002649934126205
119300000 3.9548756580833793
119400000 3.971347080284506
119500000 3.9409905619738925
119600000 3.9326598105305828
119700000 3.919873746866216
119800000 3.9406487388057143
119900000 3.927381119627976
120000000 3.911367473720173
120100000 3.9190565848857037
120200000 3.932003544062041
120300000 3.941282307032287
120400000 3.8992103172191217
120500000 3.923321364477303
120600000 3.9164575655453406
120700000 3.9113710618713364
120800000 3.929883903226323
120900000 3.926937439427038
121000000 3.924204868082897
121100000 4.090539905548887
121200000 3.959594742685586
121300000 3.912624617057946
121400000 3.908498852321781
121500000 3.9189600761051073
121600000 3.8883194980156652
121700000 3.854664341124736
121800000 3.840880716085881
121900000 3.8835373131502204
122000000 3.8585232165629506
122100000 3.849440014690876
122200000 3.8076972159514857
122300000 3.886882654713174
122400000 3.917532739930401
122500000 3.8990173402394452
122600000 3.9221063220668833
12270

...

493200000 4.587041310047319
493300000 4.216201078062302
493400000 1.553197478201068
493500000 3.868922775135338
493600000 3.4105375109158027
493700000 3.1681152034957805
493800000 3.8784432501311303
493900000 4.237560993253582
494000000 4.3883853115000235
494100000 5.053254597423802
494200000 4.167782333344887
494300000 2.780021860698988
494400000 2.698888472694685
494500000 3.710781625703467
494600000 2.6645012433164155
494700000 4.04075346492365
494800000 4.162244066721833
494900000 4.347548809977277
495000000 4.261519845152884
495100000 4.701847118293537
495200000 4.3506305622997745
495300000 1.3272735772047242
495400000 3.778275469213701
495500000 2.4201086727839245
495600000 4.037697041430205
495700000 4.120444056986789
495800000 4.122006252151525
495900000 4.0917083211868
496000000 4.192673122114357
496100000 4.502629219196078
496200000 2.7230879546478293
496300000 2.6520192670303055
496400000 2.868369342937271
496500000 3.6557329764102104
496600000 3.850197097301609
496700000 3.

521200000 3.1277116166920367
521300000 3.1864027128252324
521400000 3.31043663495789
521500000 3.3598737468691366
521600000 3.3528417656934826
521700000 3.2146361014395057
521800000 3.409402409907613
521900000 3.5234862578501804
522000000 3.4755542247709235
522100000 3.4255397893801898
522200000 3.455915438505563
522300000 3.4537310461012622
522400000 3.583822526183304
522500000 3.4665840441654225
522600000 3.3914551357931195
522700000 3.4375999606501293
522800000 3.5591103785349825
522900000 3.3663247884791496
523000000 3.4532384351629846
523100000 3.298410504883617
523200000 3.5038406321722766
523300000 3.4455618512262105
523400000 3.490091222880092
523500000 3.1260422186542156
523600000 3.074987130363014
523700000 3.120797550213523
523800000 3.2834027507279018
523900000 3.2993852756647235
524000000 3.2974522976064913
524100000 3.1396601357878944
524200000 3.3985635217447876
524300000 3.2421583370729286
524400000 3.974296534039199
524500000 3.505180825572726
524600000 4.0267044643751

548100000 3.7583653402274746
548200000 3.8066298211434018
548300000 1.1940445925680332
548400000 0.5630230110154947
548500000 0.5636546942712267
548600000 0.5664191986165742
548700000 0.5567193443449437
548800000 0.5619415916080244
548900000 0.5598771152475509
549000000 0.5622815590699216
549100000 0.5715365148273341
549200000 0.7019714244616363
549300000 0.7419723347492615
549400000 0.6671197781098385
549500000 0.5562868665557538
549600000 0.5536894102068542
549700000 0.5481108291664984
549800000 0.5502025756616692
549900000 0.5514687182832361
550000000 0.5465320500094063
550100000 0.5478245027661522
550200000 0.5581230027142267
550300000 0.5630519518011246
550400000 0.5592794335940414
550500000 0.5474636013433636
550600000 0.56391727689801
550700000 4.84363021827662
550800000 4.200058747645855
550900000 1.5381228205178512
551000000 4.286122788184468
551100000 4.78877909185941
551200000 3.363794107683066
551300000 2.7302403161350752
551400000 2.080770502745211
551500000 3.288629011601

576400000 1.0531597099363275
576500000 0.8877804690912631
576600000 0.6977611124671698
576700000 1.1032913008621037
576800000 1.0564852587305704
576900000 1.0365829681723615
577000000 1.1418222004325589
577100000 0.3536633102186885
577200000 1.3335039372985362
577300000 1.2705572192768917
577400000 0.8548885312135286
577500000 1.2749327716508212
577600000 0.7500244308369854
577700000 1.216473485935307
577800000 1.2321849576874773
577900000 1.1796821305060154
578000000 0.9021604911462466
578100000 0.6215014780024191
578200000 1.292677942720347
578300000 1.0403870490083529
578400000 1.4453648690745535
578500000 0.7611737990259396
578600000 0.8885761958751347
578700000 1.4630218626146814
best so far: 0
type: [1, 2, 1, 31, 2] 124
cases of this type: 953312
578800000 0.43570389568949736
578900000 0.2839347250212067
579000000 0.1638425941910545
579100000 0.19092407988887364
579200000 0.2817662038483123
579300000 0.21606633800554276
579400000 0.20373929451304013
579500000 0.2841032152594692
5

599800000 0.41721064348191683
599900000 0.7862018899068435
600000000 0.6709137893242539
600100000 0.7664776943271988
600200000 0.3408547183054189
600300000 0.6767283548447292
600400000 0.30599511398454504
600500000 0.7256014009510544
600600000 0.3836324815848966
600700000 0.5924384012374977
600800000 0.7540063706545996
600900000 0.444206892926544
601000000 0.7328287036618425
601100000 0.5519446325443487
601200000 0.643188632742693
601300000 0.5223859520706865
601400000 0.39104405739288867
601500000 0.5873722635327876
601600000 0.36243082398346727
601700000 0.6691142936568691
601800000 0.6479949483009875
601900000 0.5027198011385665
602000000 0.5417399401399592
602100000 0.44921342188631497
602200000 0.6878408586811755
602300000 0.5605881039976941
602400000 0.634578447746416
602500000 0.5869637585211959
602600000 0.4340881601993607
602700000 0.6354185050630867
602800000 0.5920927821176251
602900000 0.6496417938150632
603000000 0.45015801283846296
603100000 0.6191019810716378
603200000 0